# Example usage for urchin.fov
<a target="_blank" href="https://colab.research.google.com/github/VirtualBrainLab/urchin-examples/blob/main/basics/fov_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Install Urchin

Urchin is a Python package stored on PyPI, the following code needs to be run the first time you use Urchin in a Python environment. 

Urchin's full documentation can be found [on our website](https://virtualbrainlab.org/urchin/installation_and_use.html).

In [ ]:
#Installing urchin
!pip install oursin -U

## Setup Urchin and open the renderer webpage 

By default Urchin opens the 3D renderer in a webpage. Make sure pop-ups are enabled, or the page won't open properly. You can also open the renderer site yourself by replacing [ID here] with the ID that is output by the call to `.setup()` at https://data.virtualbrainlab.org/Urchin/?ID=[ID here]

Note that Urchin communicates to the renderer webpage through an internet connection, we don't currently support offline use (we hope to add support in the future).

In [ ]:
#Importing necessary libraries:
import oursin as urchin
urchin.setup()

## FOV tutorial below

### 1. Viewing a single FOV

Create a FOV

In [ ]:
fov=urchin.FOV()

Set Image for FOV

In [ ]:
fp='/mnt/c/Users/damao/Documents/spring2023/vbl/FOV_00/mpciMeanImage.images.npy' # Replace with your texture data

In [ ]:
fov.set_texture(fp)

Position FOV

In [ ]:
vertices= [[8.18, 6.97, 0.93],  # CCF coordiantes of the 4 corners for a fov,
        [8.21, 7.56, 0.87],     # roughly in the visual cortex 
        [7.62, 6.97, 0.76],
        [7.64, 7.56, 0.70]] 

In [ ]:
fov.set_position(vertices)

In [ ]:
vertices = mri2ccf('<input file path>/mpciROIs.mlapdv_estimate.npy','<output directory>')
# Alternatively, we provide helper functions to load coordinates from files
# and transform into CCF space. See mri2ccf() for more info.

Set Offset for FOV

In [ ]:
fov.set_offset(0.5)  # Move up towards brain surface

Load Brain Model

In [ ]:
urchin.ccf.load_beryl()  # View FOV relative to brain model

Delete FOV

In [ ]:
fov.delete()

### 2. Batch processing FOVs

Setup

In [ ]:
import oursin as urchin
urchin.setup(standalone=True)

In [63]:
fps=['/mnt/c/Users/damao/Documents/spring2023/vbl/FOV_00/mpciMeanImage.images.npy',
    '/mnt/c/Users/damao/Documents/spring2023/vbl/FOVDefaultTexture.png'] # Replace with your texture data

Create FOVs

In [2]:
# Create 2 FOVs
fovs=urchin.fov.create(2)

Set Image for FOVs

In [ ]:
# Set textures for the 2 FOVs
urchin.fov.set_texture(fovs,fps)

Position FOVs

In [59]:
# CCF coordiantes of the 4 corners for two fovs
vertices1= [[8.18, 6.97, 0.93],  
        [8.21, 7.56, 0.87], 
        [7.62, 6.97, 0.76],
        [7.64, 7.56, 0.70]] 
        
vertices2=[[9.74, 8.14, 1.78],
       [9.71, 8.73, 1.85],
       [9.26 , 8.14, 1.39],
       [9.24, 8.73, 1.44]]

# Or read from a file; see mri2ccf() for more details.
vertices = mri2ccf('<input file path>/mpciROIs.mlapdv_estimate.npy','<output directory>')

In [ ]:
urchin.fov.set_position(fov,[vertices1,vertices2])

Set Offset of FOVs

In [ ]:
urchin.fov.set_offset(fov,[0.5,0.5])

Load Brain Model

In [48]:
urchin.ccf.load_beryl()

Delete FOVs

In [ ]:
urchin.fov.delete(fovs)

## Helper Functions

In [21]:
bregma_ML,bregma_AP,bregma_DV = 5.7,5.4,0.33

def mri2ccf(root,saveroot,axis_orders=[0,1,2],MLrev=False,APrev=True,DVrev=True):
    """Convert coordinates from MRI transform to CCF space
    
    Parameters
    --------------
    root: str
        Input file path to read coordiantes from
    saveroot: str
        Folder to save transformed coordinates to
    axis_orders: list of three ints
        Specifies the order of ML, AP, DV axes in input data.
        Example: [0,2,1] means that column 0 is ML, column 2 is AP, and column 1 is DV.
    MLrev, APrev, DVrev: bool
        Specifies if any axis has a different direction than the CCF space.
        Example: if MLrev is set to True then the ML axis values will be multiply by -1.
    
    Returns
    --------------
    coords: list of lists of three
        Coordinates in CCF space.
    """
    MLax,APax,DVax=axis_orders
    MLrev=-1 if MLrev else 1
    APrev=-1 if APrev else 1
    DVrev=-1 if DVrev else 1

    fovcoords = np.load(root) # Will change this later

    fovcoords[:,:, 0] = MLrev*fovcoords[:,:, MLax]+bregma_ML
    fovcoords[:,:, 1] = APrev*fovcoords[:,:, APax]+bregma_AP
    fovcoords[:,:, 2] = DVrev*fovcoords[:,:, DVax]+ bregma_DV
    coords=np.array([fovcoords[0,0],fovcoords[0,-1],fovcoords[-1,0],fovcoords[-1,-1]])
    np.save(saveroot+"FOVcoordinates.bytes",coords)
    print("Coordinates:")
    print(coords)
    print(f"AP len:{fovcoords[0,0][0]-fovcoords[-1,0][0]}")
    print(f"ML len:{fovcoords[0,0][0]-fovcoords[-1,0][1]}")
    print(f"DV len:{fovcoords[0,0][2]-fovcoords[-1,0][2]}")
    print("\n")
    return coords

array([[9.73818762, 8.14252654, 1.77515527],
       [9.70847637, 8.72829154, 1.8494334 ],
       [9.2611688 , 8.14252654, 1.3904818 ],
       [9.24259926, 8.72829154, 1.43690564]])

## Old code below

In [50]:
### Code for future uses
def save_roi_sizes(root,saveroot):
    # T x N
    mpci_ROIActivityDeconvolved=np.load(root+'/mpci.ROIActivityDeconvolved.npy')
    sizes=mpci_ROIActivityDeconvolved[:num_timepoints,:num_neurons]
    print("Saving ROI sizes. \nSample data:\n")
    for x in sizes[:5,:3]:
        print(f"{x},")
    writebytes(sizes,saveroot,"ROIsizes.bytes")
    
    
def save_roi_coords(root,saveroot):
    mpciROIs_mlapdv_estimate=np.load(root+'/mpciROIs.mlapdv_estimate.npy')
    positions = mpciROIs_mlapdv_estimate[:num_neurons]
    # adjust from bregma to CCF coordinates
    # ML, AP, DV
    ccfpositions=positions
    ccfpositions[:, 0] = ccfpositions[:, 0]+bregma_ML
    ccfpositions[:, 1] = -ccfpositions[:, 1]+bregma_AP
    ccfpositions[:, 2] = -ccfpositions[:, 2] + bregma_DV
    # adjusted positions
    print("Saving ROI positions. \nSample data:\n")
    for x in ccfpositions[:3]:
        print(f"new Vector3({x[0]:.2f}f,{x[1]:.2f}f,{x[2]:.2f}f),")
    writebytes(positions,saveroot,"ROIpositions.bytes")


In [ ]:
with open('/mnt/c/Users/damao/Documents/spring2023/vbl/urchin-dev/FOV_01/FOVcoordinates.bytes','rb') as f:
    y=bytes(f.read())
arr=np.frombuffer(y,dtype=np.float32)

array([-8.5090502e-04,  2.5543182e+00,  3.5800919e-02,  2.5044539e+00,
        1.0372544e-06,  1.9718944e+00,  4.7638555e+25,  2.5533898e+00,
       -4.0475193e-19,  2.5227590e+00,  4.3603449e+35,  1.9811791e+00,
       -8.8965520e+25,  2.5394113e+00,  3.5800919e-02,  2.5044539e+00,
       -1.9193430e+15,  1.9238101e+00, -1.9007298e+34,  2.5388310e+00,
       -4.0475196e-19,  2.5227590e+00, -5.6065710e+02,  1.9296131e+00],
      dtype=float32)

In [ ]:
"""Configurations (for later)"""
AP_MIN=-5.6
AP_MAX=7.5
DV_MIN=-4
DV_MAX=4
ML_MIN=-4.8
ML_MAX=6.6

num_neurons=500
num_timepoints=10000
